In [53]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import pathlib
from sklearn import preprocessing 
import ipywidgets as widgets


# Read

In [54]:
PATH = pathlib.Path("./data")

In [55]:
weather = pd.read_csv(PATH / "Погода_2016_2020.csv")
energy = pd.read_csv(PATH / "Энергопотребление_2016_2020.csv")

# Preprocessing

In [56]:
energy.sample(3)

,StartDate,Value (kWh),day_of_week,notes
22920,2019-12-01 00:00:00,0.321,6,weekend
19289,2018-08-13 17:00:00,4.188,0,weekday
14470,2018-01-24 22:00:00,0.525,2,weekday


In [57]:
weather.sample(3)

,Date,Day,Temp_max,Temp_avg,Temp_min,Dew_max,Dew_avg,Dew_min,Hum_max,Hum_avg,Hum_min,Wind_max,Wind_avg,Wind_min,Press_max,Press_avg,Press_min,Precipit,day_of_week
774,2018-07-15,15,94,86.4,77,79,75.8,73,100,72.2,52,9,5.0,0,29.9,29.9,29.8,0.02,6
1432,2020-03-05,3,89,79.0,73,68,65.2,64,82,63.9,45,15,8.9,5,29.9,29.9,29.8,0.00,3
1352,2020-02-13,13,59,49.0,42,47,39.8,37,93,71.8,48,21,10.3,0,30.2,30.1,29.9,0.01,3


In [58]:
print(f"Weather: {weather['Date'][0]:<22} {weather['Date'][weather.shape[0] - 1]:<22} shape: {weather.shape[0]}")
print(f"Energy:  {energy['StartDate'][0]:<22} {energy['StartDate'][energy.shape[0] - 1]:<22} shape: {energy.shape[0]}\n")

Weather: 2016-01-06             2020-08-30             shape: 1552
Energy:  2016-01-06 00:00:00    2020-07-07 23:00:00    shape: 35952



In [59]:
weather["Date"] = pd.date_range("2016-06-01", "2020-08-30")
energy["StartDate"] = pd.date_range("2016-06-01 00:00:00", "2020-07-07 23:00:00", freq="1h")

In [60]:
print(f"Weather: {str(weather['Date'][0]):<22} {str(weather['Date'][weather.shape[0] - 1]):<22} shape: {weather.shape[0]}")
print(f"Energy:  {str(energy['StartDate'][0]):<22} {str(energy['StartDate'][energy.shape[0] - 1]):<22} shape: {energy.shape[0]}\n")

Weather: 2016-06-01 00:00:00    2020-08-30 00:00:00    shape: 1552
Energy:  2016-06-01 00:00:00    2020-07-07 23:00:00    shape: 35952



In [61]:
(weather["Date"].dt.day == weather["Day"]).all()

True

In [62]:
weather.sample(3)

,Date,Day,Temp_max,Temp_avg,Temp_min,Dew_max,Dew_avg,Dew_min,Hum_max,Hum_avg,Hum_min,Wind_max,Wind_avg,Wind_min,Press_max,Press_avg,Press_min,Precipit,day_of_week
490,2017-10-04,4,88,79.1,72,75,70.2,67,91,75.5,51,14,9.2,0,30.1,30.0,30.0,2.12,0
1091,2019-05-28,28,91,83.4,77,76,74.6,73,90,75.5,57,21,12.8,6,29.7,29.7,29.6,0.00,1
284,2017-03-12,12,64,55.8,48,62,49.5,44,93,80.0,62,23,12.6,0,30.1,30.0,30.0,1.37,6


In [63]:
energy.sample(3)

,StartDate,Value (kWh),day_of_week,notes
18842,2018-07-26 02:00:00,2.479,3,weekday
28521,2019-09-02 09:00:00,0.375,5,weekend
12324,2017-10-27 12:00:00,0.307,4,weekday


In [64]:
energy["notes"].unique()

array(['weekday', 'weekend', 'vacation', 'COVID_lockdown'], dtype=object)

In [65]:
energy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35952 entries, 0 to 35951
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   StartDate    35952 non-null  datetime64[ns]
 1   Value (kWh)  35952 non-null  float64       
 2   day_of_week  35952 non-null  int64         
 3   notes        35952 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(1)
memory usage: 1.1+ MB


In [66]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1552 entries, 0 to 1551
Data columns (total 19 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Date         1552 non-null   datetime64[ns]
 1   Day          1552 non-null   int64         
 2   Temp_max     1552 non-null   int64         
 3   Temp_avg     1552 non-null   float64       
 4   Temp_min     1552 non-null   int64         
 5   Dew_max      1552 non-null   int64         
 6   Dew_avg      1552 non-null   float64       
 7   Dew_min      1552 non-null   int64         
 8   Hum_max      1552 non-null   int64         
 9   Hum_avg      1552 non-null   float64       
 10  Hum_min      1552 non-null   int64         
 11  Wind_max     1552 non-null   int64         
 12  Wind_avg     1552 non-null   float64       
 13  Wind_min     1552 non-null   int64         
 14  Press_max    1552 non-null   float64       
 15  Press_avg    1552 non-null   float64       
 16  Press_

In [67]:
energy.rename({"StartDate": "Date"}, inplace=True, axis=1)

# Mergin

In [68]:
energy.shape, weather.shape

((35952, 4), (1552, 19))

In [69]:
data = pd.merge(left=energy, right=weather, how="inner", on=["Date", "day_of_week"])

In [70]:
data.shape

(1498, 21)

In [71]:
data.head(3)

,Date,Value (kWh),day_of_week,notes,Day,Temp_max,Temp_avg,Temp_min,Dew_max,Dew_avg,...,Hum_max,Hum_avg,Hum_min,Wind_max,Wind_avg,Wind_min,Press_max,Press_avg,Press_min,Precipit
0,2016-06-01,1.057,2,weekday,1,85,74.8,68,74,71.4,...,100,89.4,65,21,9.5,0,29.8,29.8,29.7,0.00
1,2016-06-02,1.281,5,weekend,2,76,71.2,66,74,70.3,...,100,96.8,89,18,7.8,0,29.8,29.8,29.7,4.33
2,2016-06-03,0.736,6,weekend,3,79,72.1,68,72,70.0,...,100,93.6,72,18,4.7,0,29.8,29.8,29.7,3.18


In [72]:
data.set_index("Date", inplace=True)

In [73]:
data.columns

Index(['Value (kWh)', 'day_of_week', 'notes', 'Day', 'Temp_max', 'Temp_avg',
       'Temp_min', 'Dew_max', 'Dew_avg', 'Dew_min', 'Hum_max', 'Hum_avg',
       'Hum_min', 'Wind_max', 'Wind_avg', 'Wind_min', 'Press_max', 'Press_avg',
       'Press_min', 'Precipit'],
      dtype='object')

# Importance exploration

In [74]:
corr = data.corr(numeric_only=True)
corr.style.background_gradient(cmap='coolwarm')

,Value (kWh),day_of_week,Day,Temp_max,Temp_avg,Temp_min,Dew_max,Dew_avg,Dew_min,Hum_max,Hum_avg,Hum_min,Wind_max,Wind_avg,Wind_min,Press_max,Press_avg,Press_min,Precipit
Value (kWh),1.000000,0.017829,-0.040117,0.602765,0.617767,0.610625,0.544454,0.558716,0.560434,0.123055,0.024436,-0.020046,0.026023,-0.133414,-0.104931,-0.341108,-0.268236,-0.029498,-0.008986
day_of_week,0.017829,1.000000,-0.002588,0.020255,0.019368,0.017839,0.020858,0.017325,0.009499,-0.001444,-0.000932,-0.003558,0.009471,0.003540,-0.030861,-0.008365,-0.020566,0.021412,-0.003220
Day,-0.040117,-0.002588,1.000000,0.005967,0.000755,-0.001977,0.002429,-0.000676,-0.006322,-0.003976,-0.005951,-0.017581,0.070071,0.048668,0.019761,-0.041066,-0.029535,0.026205,0.027902
Temp_max,0.602765,0.020255,0.005967,1.000000,0.970671,0.896819,0.853261,0.863508,0.834735,0.198014,0.009409,-0.115138,-0.017668,-0.233705,-0.222661,-0.641610,-0.538695,-0.041621,-0.034092
Temp_avg,0.617767,0.019368,0.000755,0.970671,1.000000,0.964856,0.907404,0.927545,0.902473,0.217103,0.105836,0.032589,0.029039,-0.142861,-0.135562,-0.681656,-0.577216,-0.065150,0.001527
Temp_min,0.610625,0.017839,-0.001977,0.896819,0.964856,1.000000,0.899193,0.937564,0.940381,0.221901,0.211626,0.165240,0.060535,-0.071067,-0.044836,-0.677092,-0.563582,-0.077089,0.045736
Dew_max,0.544454,0.020858,0.002429,0.853261,0.907404,0.899193,1.000000,0.970852,0.904039,0.489682,0.443943,0.334323,0.133957,-0.050420,-0.090190,-0.700926,-0.633840,-0.091189,0.075256
Dew_avg,0.558716,0.017325,-0.000676,0.863508,0.927545,0.937564,0.970852,1.000000,0.968535,0.470792,0.466556,0.365303,0.078595,-0.094989,-0.092529,-0.711440,-0.619705,-0.091366,0.073098
Dew_min,0.560434,0.009499,-0.006322,0.834735,0.902473,0.940381,0.904039,0.968535,1.000000,0.415608,0.437150,0.363220,0.026391,-0.126508,-0.090757,-0.682932,-0.560050,-0.085538,0.078316
Hum_max,0.123055,-0.001444,-0.003976,0.198014,0.217103,0.221901,0.489682,0.470792,0.415608,1.000000,0.763141,0.511610,-0.011091,-0.176681,-0.197516,-0.306529,-0.305726,-0.081099,0.124446


In [75]:
data.groupby("notes")[["Value (kWh)"]].mean()

,Value (kWh)
notes,
COVID_lockdown,0.885010
vacation,0.488231
weekday,0.979304
weekend,0.977197


In [76]:
corr_row = corr["Value (kWh)"].replace(1, 0)
(feature := corr_row.idxmax())

'Temp_avg'

The most important feature is **Temp_avg**

In [78]:
data.to_csv(PATH / "prepared_date.csv")